In [ ]:
%%writefile  MatricAdd.cu

Writing MatricAdd.cu


In [1]:
%%writefile  LayerNorm.cu
#include <iostream>
#include <cmath>
#include <cuda_runtime.h>

__global__ void LayerNorm(const float* A, float* B, int rows, int cols) {
    // Calculate row index
    int row = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < rows) {
        // Use shared memory for row-wise computation
        extern __shared__ float shared[];
        float* row_data = shared;

        // Copy row data to shared memory
        for (int col = threadIdx.y; col < cols; col += blockDim.y) {
            row_data[col] = A[row * cols + col];
        }
        __syncthreads();

        // Compute mean
        float mean = 0.0f;
        for (int col = 0; col < cols; col++) {
            mean += row_data[col];
        }
        mean /= cols;

        // Compute variance
        float variance = 0.0f;
        for (int col = 0; col < cols; col++) {
            variance += (row_data[col] - mean) * (row_data[col] - mean);
        }
        variance /= cols;
        float stddev = sqrtf(variance + 1e-7); // Add epsilon for numerical stability

        // Normalize
        for (int col = threadIdx.y; col < cols; col += blockDim.y) {
            B[row * cols + col] = (row_data[col] - mean) / stddev;
        }

        // Print statements for debugging
        if (threadIdx.y == 0) {
            printf("Row: %d, Mean: %f, Stddev: %f\n", row, mean, stddev);
        }
    }
}

int main() {
    const int rows = 10, cols = 10;
    float *A, *B;

    // Allocate host memory
    A = (float*)malloc(rows * cols * sizeof(float));
    B = (float*)malloc(rows * cols * sizeof(float));

    // Initialize input matrix
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            A[i * cols + j] = static_cast<float>(rand()) / RAND_MAX;
        }
    }

    // Allocate device memory
    float *d_a, *d_b;
    cudaMalloc(&d_a, rows * cols * sizeof(float));
    cudaMalloc(&d_b, rows * cols * sizeof(float));

    // Copy data to device
    cudaMemcpy(d_a, A, rows * cols * sizeof(float), cudaMemcpyHostToDevice);

    // Launch kernel
    int blocksize = 256;
    int gridsize = (rows + blocksize - 1) / blocksize;
    size_t shared_memory_size = cols * sizeof(float);
    LayerNorm<<<gridsize, blocksize, shared_memory_size>>>(d_a, d_b, rows, cols);

    // Synchronize device
    cudaDeviceSynchronize();

    // Copy result back to host
    cudaMemcpy(B, d_b, rows * cols * sizeof(float), cudaMemcpyDeviceToHost);

    // Print results
    printf("A:\n");
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            printf("%.2f ", A[i * cols + j]);
        }
        printf("\n");
    }

    printf("\nB:\n");
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            printf("%.2f ", B[i * cols + j]);
        }
        printf("\n");
    }

    // Free memory
    cudaFree(d_a);
    cudaFree(d_b);
    free(A);
    free(B);

    return 0;
}

Writing LayerNorm.cu


In [2]:

!nvcc LayerNorm.cu -o LayerNorm -gencode arch=compute_75,code=sm_75

!./LayerNorm

A:
0.84 0.39 0.78 0.80 0.91 0.20 0.34 0.77 0.28 0.55 
0.48 0.63 0.36 0.51 0.95 0.92 0.64 0.72 0.14 0.61 
0.02 0.24 0.14 0.80 0.16 0.40 0.13 0.11 1.00 0.22 
0.51 0.84 0.61 0.30 0.64 0.52 0.49 0.97 0.29 0.77 
0.53 0.77 0.40 0.89 0.28 0.35 0.81 0.92 0.07 0.95 
0.53 0.09 0.19 0.66 0.89 0.35 0.06 0.02 0.46 0.06 
0.24 0.97 0.90 0.85 0.27 0.54 0.38 0.76 0.51 0.67 
0.53 0.04 0.44 0.93 0.93 0.72 0.28 0.74 0.64 0.35 
0.69 0.17 0.44 0.88 0.83 0.33 0.23 0.89 0.35 0.69 
0.96 0.59 0.66 0.86 0.44 0.92 0.40 0.81 0.68 0.91 

B:
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
0.00 0.00 0.00 0.00 0.00